In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys; sys.path.append('../')
%matplotlib inline
import numpy as np
import sys,os
import pickle, gzip
import pandas as pd
import matplotlib,matplotlib.pyplot as plt
import utils
import encoders
import pytorch_models
import pandas as pd
import sklearn, sklearn.model_selection, sklearn.neighbors
import sklearn.linear_model, sklearn.ensemble
import gzip
import utils
import encoders
import collections
from mlp import MLP_train, MLP

In [3]:
import torch
import torch.optim
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
import sklearn, sklearn.metrics

In [4]:
device = "cuda"
#device = "cpu"

In [5]:
utils.btype_names

{0: 'Normal', 1: 'ESSV (PAC)', 2: 'ESV (PVC)'}

In [6]:
utils.rtype_names

{0: 'NSR', 1: 'AFib', 2: 'AFlutter'}

In [7]:
[o for o in dir(encoders) if not o.startswith("_")]

['F',
 'biosppy_mean_beat',
 'convautoencoder',
 'convautoencoder_random',
 'dir_path',
 'fft',
 'none',
 'np',
 'os',
 'pca',
 'pca_10',
 'pca_100',
 'pca_50',
 'periodogram',
 'pickle',
 'rand',
 'torch']

In [8]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('-embeddings_file', default="../test_emb_v2.csv.gz", help='File with embeddings')
parser.add_argument('-labels_file', nargs='?', default="../test_labels_v2.csv.gz", help='')
parser.add_argument('-f', help='')
parser.add_argument('-num_examples', nargs='?', type=int, default=20000, help='')
parser.add_argument('-num_trials', nargs='?', type=int, default=4, help='')
parser.add_argument('-model', type=str, default="mlp", choices=["knn","mlp","lr","adaboost","conv-resnet","conv-transformer"],help='Model to evaluate embeddings with.')
parser.add_argument('-encode_method', type=str, default=None, choices=[o for o in dir(encoders) if not o.startswith("_")], help='to encode the signals on the fly')
args = parser.parse_args()

In [9]:
args

Namespace(embeddings_file='../test_emb_v2.csv.gz', encode_method=None, f='/tmp/jupyterkjahsdaaaaaafkajsd/kernel-03aebc3e-e45b-4d94-a47f-a04d864ae005.json', labels_file='../test_labels_v2.csv.gz', model='mlp', num_examples=20000, num_trials=4)

In [10]:
def evaluate(model_name, num_examples, label_type, seed, encode_method=None):
        
    print("Generating subset", seed)

    data, labels = utils.getSubset(num_examples, 
                                   embeddings_file=args.embeddings_file, 
                                   labels_file=args.labels_file, 
                                   seed=seed,
                                   balanced=label_type)

    if encode_method != None:

        enc = getattr(encoders, encode_method)()
        print("Encoder:",enc)
        newdata = []
        for emb in data.values:
            newdata.append(enc.encode(emb))
        data = np.asarray(newdata)

    print(collections.Counter(labels[label_type]))

    X, X_test, y, y_test = \
        sklearn.model_selection.train_test_split(data, labels[label_type], 
                                                 train_size=len(labels)//2, 
                                                 test_size=len(labels)//2, 
                                                 stratify=labels[label_type],
                                                 random_state=seed)
    print("X", X.shape, "X_test", X_test.shape)
    if model_name == "knn":
        model = sklearn.neighbors.KNeighborsClassifier(n_neighbors=3)
    elif model_name == "lr":
        model = sklearn.linear_model.LogisticRegression(multi_class="auto")
    elif model_name == "adaboost":
        model = sklearn.ensemble.AdaBoostClassifier()
    elif model_name == "mlp":
        network = pytorch_models.MLP(
            in_channels=len(X.values[0]), 
            out_channels=1000,
            n_classes=len(set(y)),
            seed=seed)
        model = pytorch_models.PyTorchModel(network, device="cuda", batch_size=32, n_epoch=40, seed=seed)
    elif model_name == "conv-basic":
        network = pytorch_models.CNN(
            in_channels=1, 
            out_channels=10,
            n_layers=5,
            stride=2,
            kernel=50,
            final_layer=120,
            n_classes=len(set(y)), 
            seed=seed)
        model = pytorch_models.PyTorchModel(network, device="cuda", batch_size=32, n_epoch=40, seed=seed)
    elif model_name == "conv-resnet":
        network = pytorch_models.ResNet1D(
            in_channels=1, 
            base_filters=128, # 64 for ResNet1D, 352 for ResNeXt1D
            kernel_size=16, 
            stride=2, 
            groups=32, 
            n_block=48, 
            n_classes=len(set(y)), 
            downsample_gap=6, 
            increasefilter_gap=12, 
            use_do=True,
            seed=seed)
        model = pytorch_models.PyTorchModel(network, device="cuda", batch_size=32, n_epoch=40, seed=seed)
    else:
        print("Unknown model")
        sys.exit();
    print(model)
    model.fit(X, y.values.flatten())
    y_pred = model.predict(X_test)
    bacc = sklearn.metrics.balanced_accuracy_score(y_test.values.flatten(),y_pred)
    print("   Run {} ".format(seed) + model_name + ", label_type: {}".format(label_type) + ", Balanced Accuracy Test: {}".format(bacc)) 

    return bacc


In [11]:
results = pd.read_csv("results-bal2.csv")

In [12]:
#results = pd.DataFrame()

In [13]:
#label_type = "rtype"
for label_type in ["rtype", "btype"]:
    for num_examples in [48, 120, 1200, 12000]:
        for seed in range(0,10):
            for model_desc in ["knn+convautoencoder", "knn+periodogram", "knn+biosppy_mean_beat","knn+pca_50", "knn+fft", "knn", "mlp", "conv-resnet", "conv-basic"]:

                model_desc = model_desc.split("+")
                model = model_desc[0]
                encode_method = model_desc[1] if (len(model_desc) > 1) else None

                res = {"model":model,
                      "num_examples":int(num_examples),
                      "label_type":label_type,
                      "seed":int(seed),
                      "encode_method":encode_method}
                if (len(results)> 0) and (len(pd.merge(pd.DataFrame(res, index =[0]), results)) > 0):
                    print("already done: ", res)
                    continue;
                print("running: ", res)
                bacc = evaluate(model, num_examples=num_examples, label_type=label_type, seed=seed, encode_method=encode_method)
                res["bacc"] = bacc
                results = results.append(res, ignore_index=True)
                #results.to_csv("results-bkp.csv", index=False)
                results.to_csv("results-bal2.csv", index=False)


already done:  {'model': 'knn', 'num_examples': 48, 'label_type': 'rtype', 'seed': 0, 'encode_method': 'convautoencoder'}
already done:  {'model': 'knn', 'num_examples': 48, 'label_type': 'rtype', 'seed': 0, 'encode_method': 'periodogram'}
already done:  {'model': 'knn', 'num_examples': 48, 'label_type': 'rtype', 'seed': 0, 'encode_method': 'biosppy_mean_beat'}
already done:  {'model': 'knn', 'num_examples': 48, 'label_type': 'rtype', 'seed': 0, 'encode_method': 'pca_50'}
already done:  {'model': 'knn', 'num_examples': 48, 'label_type': 'rtype', 'seed': 0, 'encode_method': 'fft'}
already done:  {'model': 'knn', 'num_examples': 48, 'label_type': 'rtype', 'seed': 0, 'encode_method': None}
already done:  {'model': 'mlp', 'num_examples': 48, 'label_type': 'rtype', 'seed': 0, 'encode_method': None}
already done:  {'model': 'conv-resnet', 'num_examples': 48, 'label_type': 'rtype', 'seed': 0, 'encode_method': None}
already done:  {'model': 'conv-basic', 'num_examples': 48, 'label_type': 'rtyp

/home/mila/c/cohenjos/.local/lib/python3.6/site-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'torch.nn.modules.dropout.Dropout' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/mila/c/cohenjos/.local/lib/python3.6/site-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/mila/c/cohenjos/.local/lib/python3.6/site-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv1d' has changed. you can retrieve the original source c

Encoder: ConvAE
Counter({0: 12, 2: 12, 1: 12})
X (18, 100) X_test (18, 100)
KNeighborsClassifier(n_neighbors=3)
   Run 6 knn, label_type: rtype, Balanced Accuracy Test: 0.3333333333333333
running:  {'model': 'knn', 'num_examples': 48, 'label_type': 'rtype', 'seed': 6, 'encode_method': 'periodogram'}
Generating subset 6
Loading cache /network/tmp1/cohenjos/workspace/icentia-ecg/.cache/48_ac08a9d7a304b1abfe2f474f4fbec0fe_0ff4734a53c86a5960b249374ab793eb.pkl.gz
Encoder: Periodogram
Counter({0: 12, 2: 12, 1: 12})
X (18, 1025) X_test (18, 1025)
KNeighborsClassifier(n_neighbors=3)
   Run 6 knn, label_type: rtype, Balanced Accuracy Test: 0.4444444444444444
running:  {'model': 'knn', 'num_examples': 48, 'label_type': 'rtype', 'seed': 6, 'encode_method': 'biosppy_mean_beat'}
Generating subset 6
Loading cache /network/tmp1/cohenjos/workspace/icentia-ecg/.cache/48_ac08a9d7a304b1abfe2f474f4fbec0fe_0ff4734a53c86a5960b249374ab793eb.pkl.gz
Encoder: BioSPPy (sample_rate:100.0)
Counter({0: 12, 2: 12, 1

/home/mila/c/cohenjos/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


loss 1.1066333055496216 valid_bacc 0.3333333333333333
loss 0.7656499743461609 valid_bacc 0.16666666666666666
loss 0.5786595940589905 valid_bacc 0.16666666666666666
loss 0.44265827536582947 valid_bacc 0.16666666666666666
loss 0.33829841017723083 valid_bacc 0.16666666666666666
loss 0.25712937116622925 valid_bacc 0.0
loss 0.19367757439613342 valid_bacc 0.0
loss 0.1445649415254593 valid_bacc 0.0
loss 0.10724733024835587 valid_bacc 0.0
loss 0.07937402278184891 valid_bacc 0.0
loss 0.058788854628801346 valid_bacc 0.0
loss 0.04362959787249565 valid_bacc 0.0
loss 0.0324602872133255 valid_bacc 0.0
loss 0.03154207393527031 valid_bacc 0.0
loss 0.030686963349580765 valid_bacc 0.0
loss 0.029888054355978966 valid_bacc 0.0
loss 0.029137959703803062 valid_bacc 0.0
loss 0.028431233018636703 valid_bacc 0.0
loss 0.02776363492012024 valid_bacc 0.0
loss 0.027131283655762672 valid_bacc 0.0
loss 0.02653099037706852 valid_bacc 0.0
loss 0.02595999278128147 valid_bacc 0.0
loss 0.02541576512157917 valid_bacc 0.0


...

/home/mila/c/cohenjos/.local/lib/python3.6/site-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'torch.nn.modules.dropout.Dropout' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/mila/c/cohenjos/.local/lib/python3.6/site-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/mila/c/cohenjos/.local/lib/python3.6/site-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv1d' has changed. you can retrieve the original source c

Encoder: ConvAE
Counter({2: 3000, 1: 3000, 0: 3000})
X (4500, 100) X_test (4500, 100)
KNeighborsClassifier(n_neighbors=3)
   Run 9 knn, label_type: btype, Balanced Accuracy Test: 0.6791111111111111
running:  {'model': 'knn', 'num_examples': 12000, 'label_type': 'btype', 'seed': 9, 'encode_method': 'periodogram'}
Generating subset 9
Loading cache /network/tmp1/cohenjos/workspace/icentia-ecg/.cache/12000_b168344177afed53b3091ab065ff84e2_0ff4734a53c86a5960b249374ab793eb.pkl.gz
Encoder: Periodogram
Counter({2: 3000, 1: 3000, 0: 3000})
X (4500, 1025) X_test (4500, 1025)
KNeighborsClassifier(n_neighbors=3)
   Run 9 knn, label_type: btype, Balanced Accuracy Test: 0.5077777777777778
running:  {'model': 'knn', 'num_examples': 12000, 'label_type': 'btype', 'seed': 9, 'encode_method': 'biosppy_mean_beat'}
Generating subset 9
Loading cache /network/tmp1/cohenjos/workspace/icentia-ecg/.cache/12000_b168344177afed53b3091ab065ff84e2_0ff4734a53c86a5960b249374ab793eb.pkl.gz
Encoder: BioSPPy (sample_rate

 Error: Not enough beats to compute heart rate.  Writing zeros instead.
 Error: Not enough beats to compute heart rate.  Writing zeros instead.
 Error: Not enough beats to compute heart rate.  Writing zeros instead.
 Error: Not enough beats to compute heart rate.  Writing zeros instead.
 Error: Not enough beats to compute heart rate.  Writing zeros instead.
 Error: Not enough beats to compute heart rate.  Writing zeros instead.
 Error: Not enough beats to compute heart rate.  Writing zeros instead.
 Error: Not enough beats to compute heart rate.  Writing zeros instead.
 Error: Not enough beats to compute heart rate.  Writing zeros instead.
 Error: Not enough beats to compute heart rate.  Writing zeros instead.
 Error: Not enough beats to compute heart rate.  Writing zeros instead.
 Error: Not enough beats to compute heart rate.  Writing zeros instead.
 Error: Not enough beats to compute heart rate.  Writing zeros instead.
 Error: Not enough beats to compute heart rate.  Writing zeros i

 Error: Not enough beats to compute heart rate.  Writing zeros instead.
 Error: Not enough beats to compute heart rate.  Writing zeros instead.
 Error: Not enough beats to compute heart rate.  Writing zeros instead.
 Error: Not enough beats to compute heart rate.  Writing zeros instead.
 Error: Not enough beats to compute heart rate.  Writing zeros instead.
 Error: Not enough beats to compute heart rate.  Writing zeros instead.
 Error: Not enough beats to compute heart rate.  Writing zeros instead.
 Error: Not enough beats to compute heart rate.  Writing zeros instead.
 Error: Not enough beats to compute heart rate.  Writing zeros instead.
 Error: Not enough beats to compute heart rate.  Writing zeros instead.
 Error: Not enough beats to compute heart rate.  Writing zeros instead.
 Error: Not enough beats to compute heart rate.  Writing zeros instead.
 Error: Not enough beats to compute heart rate.  Writing zeros instead.
 Error: Not enough beats to compute heart rate.  Writing zeros i

loss 0.09173450328871212 valid_bacc 0.6899417184521405
loss 0.09117497863099638 valid_bacc 0.6899417184521405
loss 0.09114562331047733 valid_bacc 0.6899417184521405
loss 0.09111624985801435 valid_bacc 0.6899417184521405
loss 0.09113131770122368 valid_bacc 0.6899417184521405
loss 0.09103481751759496 valid_bacc 0.6899417184521405
loss 0.09098988445063608 valid_bacc 0.6899417184521405
   Run 9 mlp, label_type: btype, Balanced Accuracy Test: 0.6895555555555556
running:  {'model': 'conv-resnet', 'num_examples': 12000, 'label_type': 'btype', 'seed': 9, 'encode_method': None}
Generating subset 9
Loading cache /network/tmp1/cohenjos/workspace/icentia-ecg/.cache/12000_b168344177afed53b3091ab065ff84e2_0ff4734a53c86a5960b249374ab793eb.pkl.gz
Counter({2: 3000, 1: 3000, 0: 3000})
X (4500, 2049) X_test (4500, 2049)
class_weight [1. 1. 1.]
loss 1.0491082863470094 valid_bacc 0.649775123481275
loss 0.7089592540158635 valid_bacc 0.7483002011827636
loss 0.5976012343853976 valid_bacc 0.802439891916252
los

In [ ]:
results2 = results.copy().fillna("None")
del results2["seed"]
results2.groupby(["label_type", "num_examples", "model","encode_method"]).mean(["bacc"])

In [ ]:
# print(results.groupby(["label_type", "num_examples", "model"]).mean(["bacc"]).to_latex())